In [1]:
import matplotlib.pyplot as plt
import circle_fit as cf
import numpy as np
import cv2
import os

In [2]:
def image_processing(gray_img):
    blur = cv2.GaussianBlur(gray_img, (5,5), 0)
    nlm  = cv2.fastNlMeansDenoising(blur, None, 10, 7, 21)
    _, otsu = cv2.threshold(nlm, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    edge = cv2.Canny(otsu, 0, 20)
    
    return edge

In [49]:
def get_edge_points(edge_img):
    edge_points = [[j,i] for i, r in enumerate(edge_img) for j, c in enumerate(r) if c == 255]

    return edge_points

## 測試擴增Edge 試圖建立虛擬連結

### Version 1 移動有問題的點

In [45]:
def move_edge(edited_edge, edge_points, img):
    
    edited_index = []
    for index, point in enumerate(edge_points):
        x, y = point[:]
        value = img[y,x]
        
        for i in range(-1, 1):
            for j in range(-1, 1):
                if int(value) - int(img[i,j]) > 2:
                    edited_edge.append([x+j, y+i])
                    edited_index.append(index)
    
    edited_edge = np.delete(edited_edge, edited_index, axis=0)

### Versoin 2 直接增加可能的點

In [41]:
def add_edge(edited_edge, edge_points, img):
    
    for index, point in enumerate(edge_points):
        x, y = point[:]
        value = img[y,x]
        
        for i in range(-1, 1):
            for j in range(-1, 1):
                if int(value) - int(img[i,j]) > 2:
                    edited_edge.append([x+j, y+i])
                    

### Version 3 直接去除問題點

In [42]:
def del_edge(edited_edge, edge_points, img):
    
    edited_index = []
    for index, point in enumerate(edge_points):
        x, y = point[:]
        value = img[y,x]
        
        for i in range(-1, 1):
            for j in range(-1, 1):
                if int(value) - int(img[i,j]) > 2:
                    edited_index.append(index)
    
    edited_edge = np.delete(edited_edge, edited_index, axis=0)

## 編輯邊緣

In [47]:
def edit_edge(edge_points, img, funct):
    edited_edge = edge_points.copy()
    
    if funct == 0:
        move_edge(edited_edge, edge_points, img)
    elif funct == 1: 
        add_edge(edited_edge, edge_points, img)
    else:
        del_edge(edited_edge, edge_points, img)
    
    edited_edge = np.unique(edited_edge, axis=0)
    return edited_edge

In [66]:
os.chdir("/home/mj/HardDisk/Github/NTU/ARCS/img/X-Ray/Case_1/Cut")

file = "10_mean_80.bmp"
img = cv2.imread(file,0)
blur = cv2.GaussianBlur(img, (5,5), 0)
nlm = cv2.fastNlMeansDenoising(blur, None, 10, 7, 21)
_, otsu = cv2.threshold(nlm, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
edge = cv2.Canny(otsu, 0, 20)
edge_points = get_edge_points(edge)

# edited_edge = edit_edge(edge_points, img, funct=2)

print(len(edge_points))
# print(len(edited_edge))

442


In [80]:
os.chdir("/home/mj/HardDisk/Github/NTU/ARCS/img/X-Ray/Case_1/Cut")
file_1 = "10_mean_80.bmp"
img_1 = cv2.imread(file_1,0)
blur_1 = cv2.GaussianBlur(img_1, (5,5), 0)
nlm_1 = cv2.fastNlMeansDenoising(blur_1, None, 10, 7, 21)
_, otsu_1 = cv2.threshold(nlm_1, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
edge_1 = cv2.Canny(otsu_1, 0, 20)
edge_points_1 = get_edge_points(edge_1)


# file_2 = "10_mean_81.bmp"
# img_2 = cv2.imread(file_2, 0)
# blur_2 = cv2.GaussianBlur(img_2, (5,5), 0)
# nlm_2 = cv2.fastNlMeansDenoising(blur_2, None, 10, 7, 21)
# _, otsu_2 = cv2.threshold(nlm_2, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
# edge_2 = cv2.Canny(otsu_2, 0, 20)# edge_points_2 = get_edge_points(edge_2)


# 顯示圖片
# cv2.imshow("Blur 1", blur_1)
# cv2.imshow("Blur 2", blur_2)

# cv2.imshow("NLM 1", nlm_1)
# cv2.imshow("NLM 2", nlm_2)

# cv2.imshow("Otsu 1", otsu_1)
# cv2.imshow("Otsu 2", otsu_2)

# cv2.imshow("Edge 1", edge_1)
# cv2.imshow("Edge 2", edge_2)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

{(137, 85),
 (137, 86),
 (137, 87),
 (137, 88),
 (137, 89),
 (137, 90),
 (137, 91),
 (137, 92),
 (137, 93),
 (137, 94),
 (137, 95),
 (137, 96),
 (137, 97),
 (137, 98),
 (137, 99),
 (137, 100),
 (137, 101),
 (137, 102),
 (137, 103),
 (137, 104),
 (137, 105),
 (137, 106),
 (137, 107),
 (137, 108),
 (138, 82),
 (138, 83),
 (138, 84),
 (138, 109),
 (138, 110),
 (138, 111),
 (138, 112),
 (138, 113),
 (139, 80),
 (139, 81),
 (139, 114),
 (139, 115),
 (139, 116),
 (139, 117),
 (140, 78),
 (140, 79),
 (140, 118),
 (140, 119),
 (140, 120),
 (141, 76),
 (141, 77),
 (141, 121),
 (141, 122),
 (141, 123),
 (141, 124),
 (142, 73),
 (142, 74),
 (142, 75),
 (142, 125),
 (142, 126),
 (142, 127),
 (143, 71),
 (143, 72),
 (143, 128),
 (143, 129),
 (144, 70),
 (144, 71),
 (144, 130),
 (144, 131),
 (145, 68),
 (145, 69),
 (145, 132),
 (145, 133),
 (146, 66),
 (146, 67),
 (146, 134),
 (146, 135),
 (147, 64),
 (147, 65),
 (147, 136),
 (147, 137),
 (148, 62),
 (148, 63),
 (148, 137),
 (148, 138),
 (149, 61),


{(1, 2), (3, 4), (5, 6)}

## 測試多次隨機取子點擬合的分佈
假定平均 cx: 203.61381  cy: 99.17248
### 結論：無法單靠尋找 Fram 各自的 Median 或 多次 Random subsampled 做連結

In [50]:
os.chdir("/home/mj/HardDisk/Github/NTU/ARCS/img/X-Ray/Case_1/Cut")

file = "10_mean_80.bmp"
img = cv2.imread(file,0)
edge = image_processing(img)
edge_points = get_edge_points(edge)


xs = []
ys = []
for iter in range(10):
    sampled_indices = np.random.choice(range(len(edge_points)), size=int(len(edge_points)), replace=False)
    sample_points = edge_points[sampled_indices]
#     cx,cy,r,_ = cf.least_squares_circle((sample_points))
    cx = np.median(sample_points[:,0])
    cy = np.median(sample_points[:,1])
    xs.append(cx)
    ys.append(cy)

print(xs)
print(ys)

[205.0, 205.0, 205.0, 205.0, 205.0, 205.0, 205.0, 205.0, 205.0, 205.0]
[100.5, 100.5, 100.5, 100.5, 100.5, 100.5, 100.5, 100.5, 100.5, 100.5]
